In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import glob
import time
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [ ]:
class RadarDataset(Dataset):
    def __init__(self, image_dir, label_dir, S=7, B=2, C=20):
        self.S = S
        self.B = B
        self.C = C
        self.transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((128, 128)),
            transforms.ToTensor()
        ])

        image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
        label_paths = sorted(glob.glob(os.path.join(label_dir, '*.txt')))

        image_map = {os.path.splitext(os.path.basename(p))[0]: p for p in image_paths}
        label_map = {os.path.splitext(os.path.basename(p))[0]: p for p in label_paths}
        common_keys = sorted(set(image_map.keys()) & set(label_map.keys()))

        self.image_paths = [image_map[k] for k in common_keys]
        self.label_paths = [label_map[k] for k in common_keys]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("L")
        img = self.transform(img)

        label_matrix = torch.zeros((self.S, self.S, self.B * 5 + self.C))

        with open(self.label_paths[idx], 'r') as f:
            for line in f.readlines():
                cls, x, y, w, h = map(float, line.strip().split())
                i = min(int(self.S * y), self.S - 1)
                j = min(int(self.S * x), self.S - 1)
                x_cell, y_cell = self.S * x - j, self.S * y - i

                if label_matrix[i, j, 4] == 0:
                    label_matrix[i, j, 0:5] = torch.tensor([x_cell, y_cell, w, h, 1])
                    label_matrix[i, j, 5 + int(cls)] = 1

        return img, label_matrix

In [ ]:
class YOLOv1(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YOLOv1, self).__init__()
        self.S, self.B, self.C = S, B, C
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(192, 128, kernel_size=1), nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=1), nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 8 * 8, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, S * S * (C + B * 5))
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x.view(-1, self.S, self.S, self.B * 5 + self.C)

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20, lambda_coord=5, lambda_noobj=0.5):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction='sum')
        self.S = S
        self.B = B
        self.C = C
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, predictions, target):
        obj_mask = target[..., 4] == 1
        noobj_mask = target[..., 4] == 0

        # Localization loss
        box_pred = predictions[obj_mask][..., 0:4]
        box_target = target[obj_mask][..., 0:4]
        coord_loss = self.lambda_coord * self.mse(box_pred, box_target)

        # Object confidence loss
        conf_pred = predictions[obj_mask][..., 4]
        conf_target = target[obj_mask][..., 4]
        obj_conf_loss = self.mse(conf_pred, conf_target)

        # No-object confidence loss
        noobj_conf_pred = predictions[noobj_mask][..., 4]
        noobj_conf_target = target[noobj_mask][..., 4]
        noobj_conf_loss = self.lambda_noobj * self.mse(noobj_conf_pred, noobj_conf_target)

        # Classification loss
        class_pred = predictions[obj_mask][..., 5:]
        class_target = target[obj_mask][..., 5:]
        class_loss = self.mse(class_pred, class_target)

        return coord_loss + obj_conf_loss + noobj_conf_loss + class_loss

In [ ]:
def decode_predictions(preds, S=7, B=2, C=20, conf_thresh=0.2):
    batch_size = preds.shape[0]
    decoded = []

    for b in range(batch_size):
        pred = preds[b]
        boxes = []

        for i in range(S):
            for j in range(S):
                cell = pred[i, j]
                for b_idx in range(B):
                    offset = b_idx * 5
                    conf = cell[offset + 4]
                    if conf > conf_thresh:
                        x = (cell[offset + 0] + j) / S
                        y = (cell[offset + 1] + i) / S
                        w = cell[offset + 2]
                        h = cell[offset + 3]
                        class_id = torch.argmax(cell[5 + B * 5:]).item()
                        boxes.append([class_id, conf.item(), x, y, w, h])
        decoded.append(boxes)
    return decoded


def yolo_to_coco_annotations(image_dir, label_dir, class_names):
    image_files = sorted(glob.glob(os.path.join(image_dir, '*.png')))
    coco_images = []
    coco_annotations = []
    ann_id = 1

    for img_id, img_path in enumerate(image_files):
        file_name = os.path.basename(img_path)
        image = Image.open(img_path)
        width, height = image.size

        coco_images.append({
            'id': img_id,
            'file_name': file_name,
            'width': width,
            'height': height
        })

        label_path = os.path.join(label_dir, file_name.replace('.png', '.txt'))
        if not os.path.exists(label_path):
            continue

        with open(label_path, 'r') as f:
            for line in f.readlines():
                cls, x, y, w, h = map(float, line.strip().split())
                x_min = (x - w / 2) * width
                y_min = (y - h / 2) * height
                box_w = w * width
                box_h = h * height

                coco_annotations.append({
                    'id': ann_id,
                    'image_id': img_id,
                    'category_id': int(cls),
                    'bbox': [x_min, y_min, box_w, box_h],
                    'area': box_w * box_h,
                    'iscrowd': 0
                })
                ann_id += 1

    categories = [{'id': i, 'name': name} for i, name in enumerate(class_names)]
    return {
        'images': coco_images,
        'annotations': coco_annotations,
        'categories': categories
    }

def predictions_to_coco(preds_dict, image_dir):
    image_files = sorted(glob.glob(os.path.join(image_dir, '*.png')))
    results = []
    width, height = 128, 128

    for img_id, file_name in enumerate(image_files):
        preds = preds_dict.get(img_id, [])
        for pred in preds:
            cls, conf, x, y, w, h = pred
            x_min = (x - w / 2) * width
            y_min = (y - h / 2) * height
            box_w = w * width
            box_h = h * height

            results.append({
                'image_id': img_id,
                'category_id': int(cls),
                'bbox': [x_min, y_min, box_w, box_h],
                'score': float(conf)
            })
    return results



In [ ]:
def evaluate(model, loader, device, class_names, image_dir, label_dir):
    model.eval()
    all_preds = {}
    start = time.time()

    with torch.no_grad():
        for i, (imgs, _) in enumerate(tqdm(loader, desc='Evaluating')):
            imgs = imgs.to(device)
            preds = model(imgs).cpu()
            decoded = decode_predictions(preds)
            for j in range(imgs.size(0)):
                global_idx = i * loader.batch_size + j
                all_preds[global_idx] = decoded[j]

    end = time.time()
    fps = len(loader.dataset) / (end - start)
    print(f"\n✅ Inference Speed: {fps:.2f} FPS")

    # Ground Truth JSON (COCO-style) 
    gt_json = {
        "info": {
            "description": "RadDet COCO-style ground truth",
            "version": "1.0",
            "year": 2025,
            "contributor": "YourName",
            "date_created": "2025-07-11"
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": [{"id": i, "name": name} for i, name in enumerate(class_names)]
    }

    ann_id = 1
    image_id = 1

    for filename in sorted(os.listdir(label_dir)):
        if not filename.endswith('.txt'):
            continue
        img_filename = filename.replace('.txt', '.png')
        img_path = os.path.join(image_dir, img_filename)

        # Safely get image size
        with Image.open(img_path) as im:
            width, height = im.size

        gt_json["images"].append({
            "id": image_id,
            "file_name": img_filename,
            "width": width,
            "height": height
        })

        with open(os.path.join(label_dir, filename), 'r') as f:
            for line in f:
                cls, x, y, w, h = map(float, line.strip().split())
                bbox = [
                    (x - w / 2) * width,    # x_min
                    (y - h / 2) * height,   # y_min
                    w * width,              # width
                    h * height              # height
                ]
                gt_json["annotations"].append({
                    "id": ann_id,
                    "image_id": image_id,
                    "category_id": int(cls),
                    "bbox": bbox,
                    "area": bbox[2] * bbox[3],
                    "iscrowd": 0
                })
                ann_id += 1

        image_id += 1

    with open('ground_truth.json', 'w') as f:
        json.dump(gt_json, f)

    # Predictions JSON (COCO-style) 
    pred_json = []
    image_id = 1

    for idx, predictions in all_preds.items():
        img_filename = sorted(os.listdir(image_dir))[idx]
        with Image.open(os.path.join(image_dir, img_filename)) as im:
            width, height = im.size

        for pred in predictions:
            cls, conf, x, y, w, h = pred
            bbox = [
                (x - w / 2) * width,
                (y - h / 2) * height,
                w * width,
                h * height
            ]
            pred_json.append({
                "image_id": image_id,
                "category_id": int(cls),
                "bbox": bbox,
                "score": float(conf)
            })

        image_id += 1

    with open('predictions.json', 'w') as f:
        json.dump(pred_json, f)

    if len(pred_json) == 0:
        print("⚠️ No predictions above threshold. Skipping mAP evaluation.")
        return

    # Run COCO Evaluation 
    coco_gt = COCO('ground_truth.json')
    coco_dt = coco_gt.loadRes('predictions.json')
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

In [ ]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class_names = ['Rect', 'Barker', 'Frank', 'P1', 'P2', 'P3', 'P4', 'Px', 'ZadoffChu', 'LFM', 'FMCW']
    image_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/RadDet-1T-128/RadDet40k128HW001Tv2/images/test'
    label_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/RadDet-1T-128/RadDet40k128HW001Tv2/labels/test'

    # image_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/RadDet-9T-128/RadDet40k128HW009Tv2/images/test'
    # label_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/RadDet-9T-128/RadDet40k128HW009Tv2/labels/test'

    # image_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/NIST-128/NISTSpecMaxHold128Data/images/test'
    # label_dir = 'C:/Users/lenovo/OneDrive/Desktop/IP/NIST-128/NISTSpecMaxHold128Data/labels/test'

    dataset = RadarDataset(image_dir=image_dir, label_dir=label_dir)
    loader = DataLoader(dataset, batch_size=16, shuffle=False)

    model = YOLOv1().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = YoloLoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        loop = tqdm(loader, leave=False)

        for imgs, labels in loop:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs)
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_description(f"Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1} avg loss: {total_loss/len(loader):.4f}")

    evaluate(model, loader, device, class_names, image_dir, label_dir)

In [8]:
if __name__ == '__main__':
    train()

Epoch 1 avg loss: 50.8058


Evaluating: 100%|██████████| 626/626 [06:39<00:00,  1.57it/s]



✅ Inference Speed: 24.84 FPS
⚠️ No predictions above threshold. Skipping mAP evaluation.
